# This Notebook:
## SVR is good for finding a model for the data that we have but it is not good for prediction

This notebook is about hoe to use SVR(support vector regression) on forex data. The initial results seems good but there are two things that must be considered:

- The parameters optimal values are very important and if I want to continue with this approach I haev to read more about them.
- The predicted results seems to be very similar to the real test value but with a delay, I have to find a way for surpass this and get prediction for the next 2 or 3 timeslots.

# Resources:

- https://calculatedcontent.com/2012/02/06/kernels_part_1/
- https://calculatedcontent.com/2012/04/09/kernels-part-2-affine-quantum-gravity/ 
- https://calculatedcontent.com/2012/09/06/kernels-part-2-affine-quantum-gravity-continued/
- http://stackoverflow.com/questions/34475245/sklearn-svm-svr-and-svc-getting-the-same-prediction-for-every-input
- http://stackoverflow.com/questions/21393704/scikit-learn-svm-giving-me-zero-error-but-cant-predict?rq=1

# Tips:

- You should look to tune your C parameter using cross validation so that it is able to perform best on whichever metric matters most to you. You may want to look at GridSearchCV to help you do this.
- http://stackoverflow.com/questions/15775727/time-series-prediction-using-support-vector-regression?noredirect=1&lq=1
- For time series prediction I suggest building the training test set as

 x[0]=Y[0:K]; y[0]=Y[K]
 
 x[1]=Y[1:K+1]; y[1]=Y[K+1]
 ...

 that is, try to predict future elements of the sequence from a window of previous elements. 
- 

In [1]:
import sklearn
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

# Importing Data

## A better way for importing data is using 'csv' library

In [3]:
import csv

In [2]:
prices = []
dates = []

In [1]:
def get_data(filename):
    with open(filename,'r') as csvfile:
        csvFileReader = csv.reader(csvfile)
        # we read each line/row with the "next" method
        # we skip the first row because it is just heading
        next(csvFileReader)
        for row in csvFileReader:
            # we append the close price to price list
            # we can create any other list with the same syntax
            prices.append(float(row[4]))
    return

In [ ]:
#dates = np.reshape(dates, (len(dates),1))
xaxis = range(len(prices))
xaxis = np.reshape(xaxis, (len(xaxis),1))

plt.scatter(xaxis,prices,color='black', label='Data')
plt.show()

prices = []
get_data('Data/EURUSD_Candlestick_1_h_BID_09.01.2017-14.01.2017.csv')

prices = prices[:119]
prices = np.asarray(prices)
print prices.shape
print '========='

reshprices = np.reshape(prices, (len(prices),1))
print reshprices.shape
print '*****'

print 'the original value of x:'
print prices[50]

pred = predict_price(reshprices,50)
#pred = predict_price(prices,50)
print 'Predicted Prices:'
print pred

In [2]:
data = pd.read_csv('Data/EURUSD_Candlestick_1_h_BID_09.01.2017-14.01.2017.csv',delimiter=',',header=None)
npdata = np.asarray(data)

In [3]:
print data.shape
#print data.values
print npdata[0:2,:]
dataclose = npdata[1:120,4]
dataopen = npdata[1:120,1]
print dataclose[0]
print dataopen[0]

(145, 6)
[['Gmt time' 'Open' 'High' 'Low' 'Close' 'Volume']
 ['09.01.2017 00:00:00.000' '1.05294' '1.05358' '1.05224' '1.05350'
  '6551.899999999998']]
1.05350
1.05294


In [4]:
%matplotlib qt
plt.plot(dataclose,'.b',label='Close Price')
plt.plot(dataopen,'.r',label='Open Price')
plt.legend(loc='best')
plt.grid(True)
#plt.show()

# Forecasting

In [5]:
from sklearn.svm import SVR

In [10]:
ytrainclose = dataclose[:100]
ytestclose = dataclose[99:]

ytrainclose = ytrainclose.astype(np.float)
ytestclose = ytestclose.astype(np.float)

#ytrainclose = ytrainclose - 1
#ytestclose = ytestclose -1

xtrain = np.ones(len(ytrainclose))
#xtrain[0] = 1.0540
for i in range(1,len(xtrain)):
    xtrain[i] = ytrainclose[i-1]

xtest = np.ones(len(ytestclose))
for i in range(1,len(xtest)):
    xtest[i] = ytestclose[i-1]

ytrainclose = ytrainclose.reshape((100,1))
ytestclose = ytestclose.reshape((20,1))
xtrain = xtrain.reshape((100,1))
xtest = xtest.reshape((20,1))    

print ytrainclose.shape
print ytestclose.shape
print xtrain.shape
print xtest.shape

(100L, 1L)
(20L, 1L)
(100L, 1L)
(20L, 1L)


In [11]:
#Not working because C and epsilon are very different than what they should be
# I have to read about C and epsilon
clf = SVR(kernel='rbf', C=1e3, verbose=True)
#clf.fit(xtrain,ytrainclose)
pred = clf.fit(xtrain,ytrainclose).predict(xtrain)

[LibSVM]

http://stackoverflow.com/questions/34475245/sklearn-svm-svr-and-svc-getting-the-same-prediction-for-every-input

In [12]:
regsvr = SVR(C=100000, epsilon=0.00001)
pred2 = regsvr.fit(xtrain,ytrainclose).predict(xtest)

In [22]:
xtest[0] = 0.06

In [25]:
#print 'Real values: \n', ytestclose
# print 'xtest: ', xtest
# print '\n Predicated: \n', pred2
#print clf.score()
# print 'ytest: ', ytestclose

In [13]:
%matplotlib qt
#plt.plot(ytrainclose,'-b',label='Real Price')
plt.plot(ytestclose,'-b',label='Real Price')
plt.plot(pred2,'-r',label='Predicated Price')
plt.legend()
plt.grid(True)

### Linear Kernel - Bad Results

In [15]:
regsvr_lin = SVR(kernel='linear',C=1000, epsilon=0.0001)
linpred = regsvr_lin.fit(xtrain,ytrainclose).predict(xtest)

In [16]:
linpred

array([ 0.0576161 ,  0.05761262,  0.05760848,  0.05760853,  0.05760316,
        0.05760236,  0.05760109,  0.05759669,  0.05759673,  0.05759602,
        0.05759633,  0.05759625,  0.05761399,  0.05760593,  0.0576054 ,
        0.05759972,  0.05760197,  0.05759686,  0.05759985,  0.05759761])

In [17]:
%matplotlib qt

#plt.plot(ytrainclose,'-b',label='Real Price')
plt.plot(ytestclose,'-b',label='Real Price')
plt.plot(linpred,'-r',label='Predicated Price')
plt.legend()
plt.grid(True)

### Poly Kernel - Bad Results

In [18]:
regsvr_poly = SVR(kernel='poly',C=1000, epsilon=0.0001)
polypred = regsvr_poly.fit(xtrain,ytrainclose).predict(xtest)

In [19]:
polypred

array([ 0.05774002,  0.05773998,  0.05773994,  0.05773994,  0.05773988,
        0.05773987,  0.05773985,  0.0577398 ,  0.0577398 ,  0.0577398 ,
        0.0577398 ,  0.0577398 ,  0.05773999,  0.05773991,  0.0577399 ,
        0.05773984,  0.05773986,  0.05773981,  0.05773984,  0.05773981])

In [20]:
%matplotlib qt

#plt.plot(ytrainclose,'-b',label='Real Price')
plt.plot(ytestclose,'-b',label='Real Price')
plt.plot(polypred,'-r',label='Predicated Price')
plt.legend()
plt.grid(True)